# Data wrangling

This notebook consolidates the separate data sources into a single CSV file.


In [1]:
from pathlib import Path
import pandas as pd


import sys

sys.path.append("../scripts")

from shared import IVY_COLORS, ingest_bd, ingest_ill

In [2]:
all_data = []
for file in sorted(Path("../data/").glob("*.xlsx")):
    if file.name.startswith("borrow_direct"):
        bd = ingest_bd(file)
        df = pd.DataFrame()
        df[
            [
                "Date",
                "System",
                "OriginLocation",
                "DestinationLocation",
                "Type",
                "CallNumber",
            ]
        ] = bd[
            [
                "Date",
                "System",
                "Pick location",
                "Pickup location",
                "Type",
                "Local Library Call Number",
            ]
        ]
        if "borrowed" in file.name:
            df["To"] = "Dartmouth College"
            df["From"] = bd["Partner"]
        else:
            df["From"] = "Dartmouth College"
            df["To"] = bd["Partner"]
        all_data.append(df)

    if file.name.startswith("ill"):
        ill = ingest_ill(file)
        df = pd.DataFrame()
        df[["Date", "System", "Type", "CallNumber"]] = ill[
            ["Date", "System", "Type", "Call Number"]
        ]
        if "borrowed" in file.name:
            df[["From", "OriginLocation", "DestinationLocation"]] = ill[
                ["Partner", "Library Name", "Location"]
            ]
            df["To"] = "Dartmouth College"
        else:
            df[["To", "OriginLocation", "DestinationLocation"]] = ill[
                ["Partner", "Location", "Library Name"]
            ]
            df["From"] = "Dartmouth College"
        all_data.append(df)

all_data = pd.concat(all_data)

In [3]:
# Columns: Date, System, From, To, OriginLocation, DestinationLocation, Type
all_data = all_data[
    [
        "Date",
        "System",
        "From",
        "To",
        "OriginLocation",
        "DestinationLocation",
        "Type",
        "CallNumber",
    ]
]

In [5]:
all_data.to_csv("../data/derived/all_records.csv", index=None)